In [ ]:
import nipype
import os,glob,sys,shutil
sys.path.append("/usr/lib/ants/")
import nipype.interfaces.fsl as fsl
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
import nipype.interfaces.ants.legacy as antsL
import nipype.interfaces.ants as ants
from nipype.interfaces.ants import Registration
from nipype.interfaces.ants import RegistrationSynQuick
from IPython.display import Image
from nipype.caching import Memory


mem = Memory(base_dir='/data/HCP_Data/NipypeScratch/')
regScratchDir = "/data/HCP_Data/NipypeScratch/run_hcp_reg/"

In [ ]:
# """
# Setup for DataGrabber inputs needed for the registration pipeline; This is using the freesurfer nodif and t1 masks
# """
ds = nio.DataGrabber(infields=['subject_id'],
    outfields=['nodif_brain','nodif_brain_mask','struct','struct_mask','struct_brain'])

datasource = pe.Node(interface=ds,name="datasource")
# create a node to obtain the functional images
datasource.inputs.base_directory = "/data/HCP_Data/HCP_BedpostData/"
datasource.inputs.template ='*'
datasource.inputs.sort_filelist = True
datasource.inputs.field_template = dict(
    nodif_brain='%s/T1w/Diffusion/nodif_brain.nii*',
    nodif_brain_mask='%s/T1w/Diffusion/nodif_brain_mask.nii*',
    struct='%s/T1w/T1w_acpc_dc.nii*',
    struct_mask='%s/T1w/brainmask_fs.nii*', 
    struct_brain='%s/T1w/T1w_acpc_dc_masked.nii*'
)

datasource.base_dir="/data/HCP_Data/NipypeScratch/datasource_cache_1"
datasource.inputs.template_args = dict(
             nodif_brain = [['subject_id']],
             nodif_brain_mask =  [['subject_id']],
             struct =  [['subject_id']],
             struct_mask = [['subject_id']],
             struct_brain = [['subject_id']] )

In [ ]:
subjRootDir = "/data/HCP_Data/HCP_BedpostData/"
FULL_SUBJECT_LIST = [x for x in os.listdir(subjRootDir) if os.path.isdir( subjRootDir+x+'/T1w/Diffusion.bedpostX')]
print(len(FULL_SUBJECT_LIST),"Subjects are potentially available to be processed!")

In [ ]:
datasource.inputs.raise_on_empty = False
completeSubjs = []
missingSubjs = []

for s in FULL_SUBJECT_LIST:
    datasource.inputs.subject_id = s
    results = datasource.run()
    if None not in results.outputs.get().values():
        completeSubjs.append(s)
    else:
        missingSubjs.append(s)
print("Complete:",len(completeSubjs),"MISSING Scans:",len(missingSubjs))

In [ ]:
"""
Setup for Registration  Pipeline InfoSource i.e. subjects
"""
subj_infosource = pe.Node(interface=util.IdentityInterface(fields=['subject_id']),  name="subj_infosource")
#infosource.iterables = ('subject_id', SampleSubjList)
subj_infosource.iterables = ('subject_id', completeSubjs)
### Above just converts the list of subjects into an iterable list I can connect to the next part of the pipeline

In [ ]:
#### RIGID BODY REGISTRATION OF DTI -- >  Struct Brain    using RegSynQuick
reg_Struct_to_MNI = mem.cache(RegistrationSynQuick)
reg_DTI_to_Struct = mem.cache(RegistrationSynQuick)

for subjID in completeSubjs:
    subjWD = os.path.join(regScratchDir,str(subjID))
    if not os.path.isdir(subjWD):
        os.makedirs(subjWD)

    ### DTI -->  STRUCT
    struct_brain = '/data/HCP_Data/HCP_BedpostData/%s/T1w/T1w_acpc_dc_masked.nii.gz' % subjID
    nodif_brain = '/data/HCP_Data/HCP_BedpostData/%s/T1w/Diffusion/nodif_brain.nii.gz' % subjID
    MNI_1MM = '/usr/share/fsl/5.0/data/standard/MNI152_T1_1mm_brain.nii.gz'

    d_to_s_op= os.path.join(regScratchDir,str(subjID),"%s_DTI_to_Struct_" % subjID)
    s_to_MNI_1mm_op = os.path.join(regScratchDir,str(subjID),"%s_Struct_to_MNI_1mm_" % subjID)
    
    reg_DTI_to_Struct(moving_image=nodif_brain,fixed_image=struct_brain,num_threads=24,
                      transform_type='r',output_prefix=d_to_s_op)
    
    #### Struct --> MNI
    reg_Struct_to_MNI(moving_image=struct_brain,fixed_image=MNI_1MM,num_threads=24,
                     output_prefix=s_to_MNI_1mm_op)
    
    ## TO DO-- COMBINE ROIS

In [ ]:
# reg_Struct_to_MNI.inputs.transforms = ['SyN']  ## Mandatory
# reg_Struct_to_MNI.inputs.dimension = 3
#reg_Struct_to_MNI.inputs.float = False
# reg_Struct_to_MNI.inputs.interpolation = 'NearestNeighbor'### or Linear?
# reg_Struct_to_MNI.inputs.metric = ['MI']
# reg_Struct_to_MNI.inputs.metric_weight = [1.0]
# reg_Struct_to_MNI.inputs.smoothing_sigmas = [[3,0]]
# reg_Struct_to_MNI.inputs.shrink_factors = [[8,4,2,1]]
# reg_Struct_to_MNI.inputs.number_of_iterations= [[10000,10000,10000,10000]]
# reg_Struct_to_MNI.inputs.radius_or_number_of_bins=[32]
# reg_Struct_to_MNI.inputs.radius_bins_stage_trait=[1]
# reg_Struct_to_MNI.inputs.sampling_strategy = ['Regular']
# reg_Struct_to_MNI.inputs.sigma_units=['vox']
# reg_Struct_to_MNI.inputs.transform_parameters = [(0.25,)]
# reg_Struct_to_MNI.inputs.winsorize_lower_quantile = 0.005
# reg_Struct_to_MNI.inputs.winsorize_upper_quantile = 0.995
# reg_Struct_to_MNI.inputs.use_histogram_matching = False
# reg_Struct_to_MNI.inputs.num_threads = 30


In [ ]:
reg_DTI_to_Struct.cmdline
#/usr/lib/ants/ANTS 3 -m PR[MNI152_T1_1mm_brain.nii.gz,T1w_acpc_dc_masked.nii.gz,1,4] 
# -t SyN[0.25] -r Gauss[3,0] -o T1_to_MNI_ -i 30x90x20 --use-Histogram-Matching 
# --number-of-affine-iterations 10000x10000x10000x10000x10000 --MI-option 32x16000


In [ ]:
# wi = result.outputs.warped_image
# from niwidgets import NiftiWidget
# from niwidgets import examplet1

# test_widget = NiftiWidget(wi)
# test_widget.nifti_plotter()


# fixed_widget = NiftiWidget( "/data/HCP_Data/HCP_BedpostData/100206/T1w/T1w_acpc_dc_masked.nii.gz")
# fixed_widget.nifti_plotter()

In [ ]:
# antsL.antsIntroduction()
# ants.WarpImageMultiTransform()
run_hcp_reg  = pe.Workflow(name="run_hcp_reg")
run_hcp_reg.base_dir = "/data/HCP_Data/NipypeScratch/"

#samples_base_name_fxn = lambda x : x.replace('_th1samples.nii.gz','')

run_hcp_reg.connect(subj_infosource,'subject_id',datasource,'subject_id')
# ### Connect the dti_datasource to the pbx2 command
run_hcp_reg.connect( datasource,'struct_brain',regDTI_to_Struct,'reference_image')
run_hcp_reg.connect( datasource,'nodif_brain',regDTI_to_Struct,'input_image')
# runpbx2.connect( datasource,'fsamples',pbx2,'fsamples')
# runpbx2.connect( datasource,'nodif_brain_mask',pbx2,'mask')
# runpbx2.connect( datasource, ('thsamples', samples_base_name_fxn ), pbx2,'samples_base_name') ###  NOTE THIS IS A WEIRD TUPLE IS

#run_hcp_reg.run()
run_hcp_reg.run(plugin='MultiProc', plugin_args={'n_procs' : 10})

In [ ]:
### Concepts/Steps

## 3 Spaces     DTI    ---- >    T1        --->MNI
## DTI is probably "best" registered using the nodif_brain which is the first image of the "data.nii.gz" file
#antsIntroduction.sh -d 3 -r $MNI_BRAIN  -i $T1_BRAIN -o T1_to_MNI_ -t GR


In [ ]:
# DTI='/data/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/data.nii.gz'
# DTI_B0='/data/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain.nii.gz'
# DTI_M='/data/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain_mask.nii.gz'
# T1_M='/data/HCP_Data/HCP_BedpostData/100610/T1w/brainmask_fs.nii.gz'
# T1='/data/HCP_Data/HCP_BedpostData/100610/T1w/T1w_acpc_dc.nii.gz'
# # DTI_TO_T1='/data/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/nodif_to_T1_deformed.nii.gz
# MNI_BRAIN='/usr/share/fsl/5.0/data/standard/MNI152_T1_1mm_brain.nii.gz'
# #T1_BRAIN=''/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/T1w_acpc_dc_masked.nii.gz
# ROI_ONE='/data/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Left.nii.gz' 


In [ ]:

# echo $IN $T1_M $T1

# ## NOTES PROBABLY wouLD bE QUICKER IF WE USED THE T1 IMAGE WIHT THE MASK APPLIED... TO DO!!

# export PATH=$PATH:/usr/lib/ants
# export ANTSPATH=/usr/lib/ants

# ## Get the first B0 image from the DTI data set
# #fslroi $DTI $DTI_B0 0 1 


# ## Register the B0/DTI image to the T1 --- we perhaps should have used the masked/brainonly T1??
# #antsIntroduction.sh -d 3 -r $T1 -i $DTI_B0 -o nodif_to_T1_ -t RI


# ## Need to apply the brain mask to the T1 image to speed things up
# #fslmaths $T1 -mas $T1_M $T1_MASKED


# ### Thsi registers the T1 Brain image nonlinearly to MNI Space.. and also generated the inverse warp
# #antsIntroduction.sh -d 3 -r $MNI_BRAIN  -i $T1_BRAIN -o T1_to_MNI_ -t GR

# ### note T1_To_MNI_Warp is the warp transform.. and we add two affine matrices which are also real files
# #WarpImageMultiTransform 3 $DTI_B0  nodif_to_MNI_test.nii.gz -R $MNI_BRAIN  T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt nodif_to_T1_Affine.txt


# ### THIS GOES IN REVERSE
# WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI.nii.gz -R $DTI_B0 -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz
# WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI_NN.nii.gz -R $DTI_B0 --use-NN -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz

# ## --use-NN: Use Nearest Neighbor Interpolation. 
 

END PRODUCT:   We will create a directory in Diffusion or maybe ROIs and maybe a subdirectory called DTI
    
    so /{SubjID}/T1w/ROIs/DTI_Space/Human_L_Hypothalamus.nii
       /{SubjID}/xfms/<<PUT XFMS HERE>> 
        

In [ ]:

# #
# #Then check that nodif-to-MNI mappings look good: slicesdir –p MNI_template.nii.gz Subj_*_nodif_to_MNI.nii.gz
# #Apply INVERSE transformation from MNI-space ROIs to individual subjects’ diffusion space
# #WarpImageMultiTransform 3 template_ROI.nii.gz SubjX_ROI.nii.gz –R nodif.nii.gz –i DTI_to_T1_Affine.txt –i T1_to_MNI_Affine.txt T1_to_MNI_InverseWarp.nii.gz

 

# #Run probtrackx using native-space ROIs… this will produce a native-space fdt_paths.nii.gz file.  Threshold and normalize in native space.

 

# #Apply transformations to get thresholded, normalized tracts into MNI space

# #WarpImageMultiTransform 3 fdt_paths_thresh_norm.nii.gz SubjX_fdt_paths_thresh_norm_in_MNIspace.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

 

# #Fslmerge –t Subj*_fdt_paths_thresh_norm_in_MNIspace.nii.gz à this becomes the input for randomi


### Start with the MNI-space hypothalamus / basal forebrain ROIs à want to end up with tractography in all subjects in MNI space

 

Compute rigid body transform from diffusion space to T1

antsIntroduction.sh -d 3 -r T1.nii.gz -i nodif.nii.gz -o nodif_to_T1_ -t RI

 

Compute nonlinear warp from T1 to MNI

antsIntroduction.sh -d 3 -r MNI.nii.gz -i T1.nii.gz -o T1_to_MNI_ -t GR

(I think GR (greedy Syn) should be faster than SY (Syn) which is the default and apparently I’ve been using GR with good results so far)

 

Apply transformations so you can check that it all looks OK

WarpImageMultiTransform 3 nodif.nii.gz nodif_to_MNI.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

Then check that nodif-to-MNI mappings look good: slicesdir –p MNI_template.nii.gz Subj_*_nodif_to_MNI.nii.gz

 

Apply INVERSE transformation from MNI-space ROIs to individual subjects’ diffusion space

WarpImageMultiTransform 3 template_ROI.nii.gz SubjX_ROI.nii.gz –R nodif.nii.gz –i DTI_to_T1_Affine.txt –i T1_to_MNI_Affine.txt T1_to_MNI_InverseWarp.nii.gz

 

Run probtrackx using native-space ROIs… this will produce a native-space fdt_paths.nii.gz file.  Threshold and normalize in native space.

 

Apply transformations to get thresholded, normalized tracts into MNI space

WarpImageMultiTransform 3 fdt_paths_thresh_norm.nii.gz SubjX_fdt_paths_thresh_norm_in_MNIspace.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

 

Fslmerge –t Subj*_fdt_paths_thresh_norm_in_MNIspace.nii.gz à this becomes the input for randomise

 

In [ ]:

thisfolder=${PWD}/sampleOutput
sub=11111 

antsRegistration --dimensionality 3 --float 0 \
--output [$thisfolder/NN_DTI_to_Struct_${sub}_,$thisfolder/NN_DTI_to_Struct_${sub}_Warped.nii.gz] \
--interpolation NearestNeighbor \
--winsorize-image-intensities [0.005,0.995] \
--use-histogram-matching 0 \
--transform Rigid[0.1] \
--metric MI[$T1_BRAIN,$DTI_B0,1,32,Regular,0.25] \
--convergence [1000x500x250x100,1e-6,10] \
--shrink-factors 8x4x2x1 \
--smoothing-sigmas 3x2x1x0vox 

#reg.inputs.convergence_threshold = [1.e-6]
#reg.inputs.convergence_window_size = [10]
# reg.inputs.use_histogram_matching = False
# reg.inputs.winsorize_upper_quantile = 0.995
# reg.inputs.winsorize_lower_quantile = 0.005

# >>> reg.inputs.metric_weight = [1]*2 # Default (value ignored currently by ANTs)
# >>> reg.inputs.radius_or_number_of_bins = [32]*2
# >>> reg.inputs.sampling_strategy = ['Random', None]
# >>> reg.inputs.sampling_percentage = [0.05, None]
# >>> reg.inputs.convergence_window_size = [20]*2
# >>> reg.inputs.sigma_units = ['vox'] * 2
# >>> reg.inputs.use_estimate_learning_rate_once = [True, True]
# >>> reg.inputs.use_histogram_matching = [True, True] # This is the default
# >>> reg.inputs.output_warped_image = 'output_warped_image.nii.gz'
# >>> reg.inputs.write_composite_transform = True
# >>> reg.inputs.collapse_output_transforms = False
# >>> reg.inputs.initialize_transforms_per_stage = False

# >>> reg.cmdline
# 'antsRegistration --collapse-output-transforms 0 --dimensionality 3 --initial-moving-transform [ trans.mat, 0 ] --initialize-transforms-per-stage 0 --interpolation Linear --output [ output_, output_warped_image.nii.gz ] --transform Affine[ 2.0 ] --metric Mattes[ fixed1.nii, moving1.nii, 1, 32, Random, 0.05 ] --convergence [ 1500x200, 1e-



In [ ]:
DTI=/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/data.nii.gz
DTI_B0=/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain.nii.gz
DTI_M=/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain_mask.nii.gz
T1_M=/HCP_Data/HCP_BedpostData/100610/T1w/brainmask_fs.nii.gz
T1=/HCP_Data/HCP_BedpostData/100610/T1w/T1w_acpc_dc.nii.gz
DTI_TO_T1=/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/nodif_to_T1_deformed.nii.gz
MNI_BRAIN=/usr/share/fsl/5.0/data/standard/MNI152_T1_1mm_brain.nii.gz 

T1_BRAIN=/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/T1w_acpc_dc_masked.nii.gz

ROI_ONE=/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Left.nii.gz  


echo $IN $T1_M $T1

## NOTES PROBABLY wouLD bE QUICKER IF WE USED THE T1 IMAGE WIHT THE MASK APPLIED... TO DO!!

export PATH=$PATH:/usr/lib/ants
export ANTSPATH=/usr/lib/ants

## Get the first B0 image from the DTI data set
#fslroi $DTI $DTI_B0 0 1 


## Register the B0/DTI image to the T1 --- we perhaps should have used the masked/brainonly T1??
#antsIntroduction.sh -d 3 -r $T1 i $DTI_B0 -o nodif_to_T1_ -t RI


## Need to apply the brain mask to the T1 image to speed things up
#fslmaths $T1 -mas $T1_M $T1_MASKED


### Thsi registers the T1 Brain image nonlinearly to MNI Space.. and also generated the inverse warp
#antsIntroduction.sh -d 3 -r $MNI_BRAIN  -i $T1_BRAIN -o T1_to_MNI_ -t GR

### note T1_To_MNI_Warp is the warp transform.. and we add two affine matrices which are also real files
#WarpImageMultiTransform 3 $DTI_B0  nodif_to_MNI_test.nii.gz -R $MNI_BRAIN  T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt nodif_to_T1_Affine.txt

### THIS GOES IN REVERSE
WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI.nii.gz -R $DTI_B0 -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz
WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI_NN.nii.gz -R $DTI_B0 --use-NN -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz

## --use-NN: Use Nearest Neighbor Interpolation. 
 
#
#Then check that nodif-to-MNI mappings look good: slicesdir –p MNI_template.nii.gz Subj_*_nodif_to_MNI.nii.gz
#Apply INVERSE transformation from MNI-space ROIs to individual subjects’ diffusion space
#WarpImageMultiTransform 3 template_ROI.nii.gz SubjX_ROI.nii.gz –R nodif.nii.gz –i DTI_to_T1_Affine.txt –i T1_to_MNI_Affine.txt T1_to_MNI_InverseWarp.nii.gz
 

#Run probtrackx using native-space ROIs… this will produce a native-space fdt_paths.nii.gz file.  Threshold and normalize in native space.


#Apply transformations to get thresholded, normalized tracts into MNI space

#WarpImageMultiTransform 3 fdt_paths_thresh_norm.nii.gz SubjX_fdt_paths_thresh_norm_in_MNIspace.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

 

#Fslmerge –t Subj*_fdt_paths_thresh_norm_in_MNIspace.nii.gz à this becomes the input for randomi

